In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import mean_squared_error

from data.city.load_cities import CITY
from data.prediction.methods.mean_predictor import PredictByMean

In [24]:
TRAIN_SIZE = 0.7
method = PredictByMean(city=CITY, train_size=TRAIN_SIZE)
method.train()

station = method.df_dataset.columns[0]
r = method.df_dataset[station].iloc[:168]
method.predict(station, r)

date
2016-04-01 00:00:00    0.112616
2016-04-01 01:00:00    0.127410
2016-04-01 02:00:00    0.126759
2016-04-01 03:00:00    0.170924
2016-04-01 04:00:00    0.305471
                         ...   
2016-04-07 19:00:00    0.121051
2016-04-07 20:00:00    0.120529
2016-04-07 21:00:00    0.115656
2016-04-07 22:00:00    0.128104
2016-04-07 23:00:00    0.110936
Name: Moyenne, Length: 168, dtype: float64

In [5]:
model = PredictByMean(CITY)
model.train()

start_week = 168 * 3
station = 1

df_predict: pd.DataFrame = model.predict(CITY.df_hours.columns[station]).reset_index(drop=True)
df_predict[CITY.df_hours.columns[station] + '-obs'] = CITY.df_hours[CITY.df_hours.columns[station]].iloc[start_week:start_week + len(df_predict)].reset_index(drop=True)
df_predict.index = CITY.df_hours['date'].iloc[start_week:start_week + len(df_predict)]

print('MSE:', mean_squared_error(df_predict[df_predict.columns[1]], df_predict[df_predict.columns[0]]))

fig = px.line(
    data_frame=df_predict
)

fig.update_layout(hovermode='x')

fig.show()

MSE: 0.07188253785881304


In [2]:
DATE_RANGE = ('04-01-2016', '10-31-2016')

df = pd.read_csv(
    f'./data/city/Toulouse/X_hour_toulouse.csv',
    index_col=0
)
df['date'] = pd.date_range(*DATE_RANGE, freq='1h')[:len(df)]
df.set_index('date', inplace=True)

In [4]:
a = pd.Series([False, True, True, False, True], index=[0, 1, 2, 9, 10])
a[0]

False

In [3]:
def get_interpolated_indices(serie: pd.Series, tolerance: float=1e-3, output_type: str='mask'):

    mask, current_mask = [], []
    for index in range(len(serie)):
        # Si < 2 éléments d'affilé de suite
        if len(current_mask) < 2:
            current_mask.append(serie.index.values[index])
            continue
        
        # On check si la différence est la même entre trois éléments d'affilé de suite
        if np.abs((serie.values[index - 2] - serie.values[index - 1]) - (serie.values[index - 1] - serie.values[index])) < tolerance:
            current_mask.append(serie.index.values[index])
            continue
        
        # Si la différence est constante et la longueur de l'interpolation < 24 heures
        if len(current_mask) < 24 and np.abs(serie.values[index - 2] - serie.values[index - 1]) < tolerance:
            current_mask = []

        if len(current_mask) > 2:
            mask.append(current_mask[1:-1])
        current_mask = [serie.index.values[index - 1], serie.index.values[index]]

    if output_type == 'mask':
        return serie[[index_mask for row in mask for index_mask in row]].index

    if output_type == 'list':
        return mask
    
    raise ValueError('The "output_type" value must be one of the following : "mask", "list".')

In [33]:
index_column = 0
tmp = df.loc[:].iloc[:, index_column].copy()
# tmp = pd.Series([-2, 1, 2, 3, 84, 56, 98, 5, 4, 3, 2, 92])

index = get_interpolated_indices(tmp, output_type='list')
print(len(index))
index = [u for v in index for u in v]
tmp.loc[index] = np.nan

a = tmp.to_frame()
a['origine'] = df.iloc[:, index_column].copy()
fig = px.line(a).show()

NameError: name 'df' is not defined